### ***Initializing the libraries***

In [2]:
import MetaTrader5 as mt5
import pandas as pd
import ta
from datetime import datetime

### ***Correcting the time error in (version_2) by initializing the mt5 first***

In [ ]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

boom = mt5.copy_rates_from_pos("Boom 1000 Index", mt5.TIMEFRAME_M1, 0, 1000)
df1 = pd.DataFrame(boom)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
print(df1)

### ***Applying the corrections to the function***

In [85]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime

def get_historical_data(symbol, timeframe, number_of_data=1000):
    if not mt5.initialize():
        print("initialize() failed")
        mt5.shutdown()
        return None
    
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, number_of_data)

    if rates is None:
        print("Failed to retrieve historical data.")
        mt5.shutdown()
        return None

    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df = df.set_index("time")
    pd.set_option("display.max_columns", None)
    pd.set_option("display.max_rows", None)
    mt5.shutdown()

    return df

if __name__ == "__main__":
    symbol = "Boom 1000 Index"
    timeframe = mt5.TIMEFRAME_M1
    data_length = 1000

    historical_data = get_historical_data(symbol, timeframe, data_length)

    if historical_data is not None:
        print(historical_data.head())
    else:
        print("Failed to retrieve historical data.")

                          open       high        low      close  tick_volume  \
time                                                                           
2023-07-30 22:08:00  10980.732  10980.732  10980.118  10980.118           59   
2023-07-30 22:09:00  10980.116  10980.116  10979.462  10979.462           58   
2023-07-30 22:10:00  10979.453  10979.453  10978.700  10978.700           58   
2023-07-30 22:11:00  10978.700  10985.779  10978.298  10985.459           60   
2023-07-30 22:12:00  10985.451  10985.451  10984.713  10984.713           59   

                     spread  real_volume  
time                                      
2023-07-30 22:08:00    1090            0  
2023-07-30 22:09:00    1090            0  
2023-07-30 22:10:00    1090            0  
2023-07-30 22:11:00    1090            0  
2023-07-30 22:12:00    1090            0  


### ***Documentation for Maverick RSI Sell Strategy***

***Title:*** RSI Sell Strategy using MetaTrader 5 and Python

1. Introduction:
   The RSI Sell Strategy is a simple automated trading approach that utilizes the Relative Strength Index (RSI) indicator to identify potential sell trade opportunities. The strategy aims to capitalize on overbought conditions in the market when the RSI value is greater than 70. It executes a sell trade when specific conditions are met, indicating a potential reversal in price direction.

2. Prerequisites:
   - MetaTrader 5 Terminal: The strategy requires the MetaTrader 5 terminal to access historical price data and execute trades through the MetaTrader 5 API.
   - Python Environment: Ensure that you have Python installed, along with the required libraries such as pandas, ta, and MetaTrader5.

3. Strategy Components:
   - get_historical_data(symbol, timeframe, number_of_data): This function retrieves historical price data for a specified symbol and timeframe from MetaTrader 5 and returns the data in a pandas DataFrame.

   - calculate_rsi(df, period): This function calculates the RSI indicator for a given DataFrame using the specified period and adds the RSI values as a new column to the DataFrame.

   - find_filling_mode(symbol): This function finds the appropriate filling mode for executing trades based on the symbol's market conditions.

   - execute_sell_trade(df, symbol, lot_size): This function executes a sell trade if specific conditions are met, based on RSI and price action.

   - run_strategy(symbol, timeframe, lot_size, data_length, period): The main function of the strategy, which runs in an infinite loop. It continuously checks for potential sell trade opportunities, calculates the RSI, and executes sell trades if conditions are met.

4. Strategy Execution:
   - The strategy starts by initializing the MetaTrader 5 terminal and connecting to the trading account.

   - It retrieves historical price data for a specified symbol and timeframe using the get_historical_data function.

   - The RSI indicator is calculated using the calculate_rsi function, with the default period set to 14.

   - The execute_sell_trade function is called to check for potential sell trade opportunities. If the RSI value is greater than 70 and specific price action conditions are met, a sell trade is executed using the MetaTrader 5 API.

   - The strategy continues to run in a true loop, checking for opportunities at regular intervals.

   - If any errors occur during the execution of the strategy, they are caught and displayed as error messages.

5. Usage:
   - Replace the "symbol" variable in the main section with your desired trading instrument (e.g., currency pair, stock, index).
   - Adjust the "timeframe" variable to your preferred timeframe for historical data retrieval (e.g., mt5.TIMEFRAME_M1 for 1-minute data).
   - Set the "lot_size" variable to your desired trade size for each sell trade.
   - Specify the "data_length" to control how much historical data the strategy retrieves for analysis.
   - The "period" variable determines the period used to calculate the RSI indicator.

6. Risk and Considerations:
   - This strategy is a simple example and may not be suitable for real trading without proper risk management and additional analysis.
   - Always conduct thorough testing and backtesting before deploying any automated trading strategy.
   - Consider incorporating stop-loss and take-profit levels to manage risk and exit positions in case of adverse price movements.

7. Conclusion:
   The RSI Sell Strategy provides a basic foundation for an automated trading approach using the RSI indicator in conjunction with price action conditions. Traders can further enhance and customize the strategy based on their specific requirements and risk appetite. Remember that successful trading strategies often require continuous monitoring, optimization, and adaptation to changing market conditions.

## ***`Final Version of the Strategy`***

In [1]:
import MetaTrader5 as mt5
import pandas as pd
import ta.momentum as momentum
from datetime import datetime, timedelta
import time 

def get_historical_data(symbol, timeframe, number_of_data = 1000):
    # Initialize the MetaTrader 5 terminal
    if not mt5.initialize():
        print("initialize() failed")
        mt5.shutdown()
        return None

    # Retrieve historical data from MetaTrader 5
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, number_of_data)

    # Check if data is retrieved successfully
    if rates is None:
        print("Failed to retrieve historical data.")
        mt5.shutdown()
        return None

    # Transform tuples into a DataFrame
    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit= "s")
    df = df.set_index("time")

    # Shutdown the MetaTrader 5 terminal
    mt5.shutdown()

    return df

def calculate_rsi(df, period = 14):
    try:
        # Use 'window' instead of 'n'
        rsi_indicator = momentum.RSIIndicator(df["close"], window = period)
        df["rsi"] = rsi_indicator.rsi()
    except Exception as e:
        print("Error calculating RSI:", str(e))

def execute_sell_trade(df, symbol, lot_size = 0.2):
    # Get the last row (most recent bar) from the historical data
    current_bar = df.iloc[-1]
    previous_bar = df.iloc[-2]
    
    # Check if RSI is greater than 70 (overbought) and the current close 
    # is lower than the open of the current bar, and the previous close
    # was higher than the open of the previous bar
    if current_bar["rsi"] > 70 and current_bar["close"] < current_bar["open"] \
    and previous_bar["close"] > previous_bar["open"]:
        # Initialize the MetaTrader 5 terminal
        if not mt5.initialize():
            print("initialize() failed")
            mt5.shutdown()
            return

        # Execute a sell trade
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot_size,
            "type": mt5.ORDER_TYPE_SELL,
            "price": mt5.symbol_info_tick(symbol).bid,
            "deviation": 10,
            "magic": 0,
            "comment": "RSI Sell Strategy",
            "type_filling": find_filling_mode(symbol),
            "type_time": mt5.ORDER_TIME_GTC
        }
        result = mt5.order_send(request)
        print("Sell Trade Executed:", result.comment)

        # Shutdown the MetaTrader 5 terminal
        mt5.shutdown()

def find_filling_mode(symbol):
    for i in range(2):
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": mt5.symbol_info(symbol).volume_min,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "type_filling": i,
            "type_time": mt5.ORDER_TIME_GTC
        }
        result = mt5.order_check(request)
        if result.comment == "Done":
            break
    return i

def run_strategy(symbol, timeframe, lot_size=0.1, data_length = 1000, period = 14):
    while True:
        try:
            # Retrieve historical data
            df = get_historical_data(symbol, timeframe, data_length)

            # Check if the historical data is retrieved successfully
            if df is not None:
                # Calculate RSI
                calculate_rsi(df, period)
                
                # Execute sell trade if conditions are met
                execute_sell_trade(df, symbol, lot_size)

        except Exception as e:
            print("Error executing the strategy:", str(e))

        # Wait for some time before checking for opportunities again
        # Adjust the sleep duration as per your preference (in seconds)
        sleep_duration = 60  # Wait for 1 minute
        print(f"Waiting for {sleep_duration} seconds before checking again...")
        time.sleep(sleep_duration)

# Example usage:
if __name__ == "__main__":
    symbol = "Boom 1000 Index" # Replace with your desired symbol
    timeframe = mt5.TIMEFRAME_M1  # Replace with your desired timeframe
    lot_size = 0.2  # Replace with your desired lot size
    data_length = 1000  # Replace with the number of data points to retrieve
    period = 14  # Replace with the RSI period

    run_strategy(symbol, timeframe, lot_size, data_length, period)

Waiting for 60 seconds before checking again...
